In [1]:
include("../path.jl")
include("../ToolBox/ToolBox.jl")
import JLD
import FileIO
using .ToolBox
using .Geometry
using Colors

In [2]:
master_view = [2000:6000,1000:9000]
small = 5001:8001

5001:8001

In [3]:
# load meta and pod
master_meta = Load.slc_meta(master_meta_path);
master_precise_orbit = Load.precise_orbit(master_pod_path,master_meta["t_0"]);
slave_meta = Load.slc_meta(slave_meta_path);
slave_precise_orbit = Load.precise_orbit(slave_pod_path,slave_meta["t_0"]);

meta = (master_meta,slave_meta)
precise_orbit = (master_precise_orbit,slave_precise_orbit);

In [4]:
# load DEM
footprint = SlcUtil.footprint(master_meta, master_view)
latlon_window = ((minimum(footprint[1]), maximum(footprint[1])), (minimum(footprint[2]), maximum(footprint[2])))
dem = Load.dem(dem_path, latlon_window; nan_fill= 0, padding=[90,90]);

In [ ]:
slave_data,flat,lut =  coregister_slave(master_view,slave_data_path,meta,precise_orbit,dem)
slave_data,mosaic_view = SlcUtil.mosaic(slave_data,master_view,meta[1]);
flat,mosaic_view = SlcUtil.mosaic(flat,master_view,meta[1]);

master_data = Load.slc_data(master_data_path,master_view);
master_data,mosaic_view = SlcUtil.mosaic(master_data,master_view,meta[1]);

In [ ]:
kernel = ones(4,14)
complex_coherence, master_intensity, slave_intensity, lines, samples = SlcUtil.complex_coherence(master_data, slave_data, flat, kernel, mosaic_view);

In [ ]:
coherence_img = Gray.(abs.(complex_coherence[1:2:end,1:7:end]))

In [ ]:
#FileIO.save("Coherence_multiloock_coherence_sub.jpg",coherence_img)

In [ ]:
phase_img = SlcUtil.plot_phase(complex_coherence[1:2:end,1:7:end])

In [ ]:
#FileIO.save("Coherence_multiloock_phase_sub.jpg",phase_img)

In [ ]:
intensity = SlcUtil.show_img(master_intensity[1:3:end,1:10:end].-minimum(minimum(master_intensity[1:2:end,1:7:end])))

In [ ]:
#FileIO.save("Multiloock_master_intensity_sub.jpg",intensity)

In [ ]:
JLD.save("inten_VV.jld", "data", master_intensity[1:2:end,1:7:end].-minimum(minimum(master_intensity[1:2:end,1:7:end])))
JLD.save("Coherence_VV.jld", "data", abs.(complex_coherence[1:2:end,1:7:end]))